In [18]:
from docplex.mp.model import Model
import random
import plotly.graph_objects as go
import plotly.express as px

**Conjuntos**

$$ \text{CLIENTES} = \{1, 2, \ldots, N\} $$
$$ \text{NODOS} = \{0\} \cup \text{CLIENTES} $$
$$ \text{CAMIONES} = \{1, 2, 3, 4, 5, 6, 7\} $$

In [19]:
# Conjuntos
N = 20  # Número de clientes (ajusta según sea necesario)
CLIENTES = list(range(1, N + 1))
NODOS = [0] + CLIENTES
CAMIONES = [1, 2, 3, 4, 5, 6, 7]

In [20]:
random.seed(62)

**Parámetros**

$$ \text{DISTANCIAS}_{n1,n2}: \text{Distancia euclidiana entre } n1 \in \text{NODOS} \text{ y } n2 \in \text{NODOS} $$
$$ \text{TIEMPO\_SERVICIO}_{cliente}: \text{Tiempo de servicio en el cliente } cliente \in \text{CLIENTES} $$
$$ \text{VENTANA\_TIEMPO}_{cliente} = [a_{cliente}, b_{cliente}]: \text{Ventana de tiempo para el cliente } cliente \in \text{CLIENTES} $$
$$ \text{TIEMPO\_TRAVEL}_{n1,n2}: \text{Tiempo de viaje entre } n1 \in \text{NODOS} \text{ y } n2 \in \text{NODOS} $$

In [21]:
# Parámetros
COORDENADAS = {n: {"x": random.randint(0, N * 10), "y": random.randint(0, N * 10)} for n in NODOS}
VEL_CAMION = 10
TIEMPO_VIAJE = {(n1, n2): (((COORDENADAS[n1]["x"] - COORDENADAS[n2]["x"]) ** 2 + (COORDENADAS[n1]["y"] - COORDENADAS[n2]["y"]) ** 2) ** 0.5) / VEL_CAMION for n1 in NODOS for n2 in NODOS}
TIEMPO_SERVICIO = {cliente: random.randint(3, 7) for cliente in CLIENTES}
VENTANA_TIEMPO = {
    1: {"mint": 0, "maxt": 56},
    2: {"mint": 3, "maxt": 75},
    3: {"mint": 10, "maxt": 55},
    4: {"mint": 49, "maxt": 93},
    5: {"mint": 3, "maxt": 49},
    6: {"mint": 19, "maxt": 63},
    7: {"mint": 2, "maxt": 57},
    8: {"mint": 13, "maxt": 61},
    9: {"mint": 23, "maxt": 65},
    10: {"mint": 4, "maxt": 90},
    11: {"mint": 0, "maxt": 65},
    12: {"mint": 10, "maxt": 52},
    13: {"mint": 20, "maxt": 64},
    14: {"mint": 31, "maxt": 76},
    15: {"mint": 2, "maxt": 30},
    16: {"mint": 0, "maxt": 82},
    17: {"mint": 21, "maxt": 56},
    18: {"mint": 44, "maxt": 86},
    19: {"mint": 5, "maxt": 15},
    20: {"mint": 27, "maxt": 72},
}

In [22]:
fig = go.Figure()

# Añadir punto de depósito en rojo
fig.add_trace(
    go.Scatter(
        x=[COORDENADAS[0]["x"]],
        y=[COORDENADAS[0]["y"]],
        mode="markers",
        name="Depósito",
        marker=dict(color="red", size=15),
    )
)

# Añadir puntos de clientes en negro
fig.add_trace(
    go.Scatter(
        x=[coord["x"] for coord in list(COORDENADAS.values())[1:]],
        y=[coord["y"] for coord in list(COORDENADAS.values())[1:]],
        mode="markers",
        name="Clientes",
        marker=dict(color="black", size=10),
    )
)

# Añadir anotaciones de clientes
for cliente in CLIENTES:
    coord_cliente_x = COORDENADAS[cliente]["x"]
    coord_cliente_y = COORDENADAS[cliente]["y"]
    ventana = f"[{VENTANA_TIEMPO[cliente]['mint']}, {VENTANA_TIEMPO[cliente]['maxt']}]"
    fig.add_annotation(x=coord_cliente_x - 8, y=coord_cliente_y + 8, text=f"<b>C{cliente}: </b>{ventana}", showarrow=False, font=dict(size=10))

# Configurar el layout del gráfico
fig.update_layout(
    title=f"COORDENADAS NODOS",
    xaxis_title="X",
    yaxis_title="Y",
    width=800,
    height=600,
    template="ggplot2",
)

fig.show()

In [23]:
# Creación del modelo
model = Model("VRPTW")

**Variables de Decisión**
$$ x_{n1, n2, camion} = \begin{cases} 
1 & \text{Si el camión } camion \text{ viaja de } n1 \in \text{NODOS} \text{ a } n2 \in \text{NODOS} \\ 
0 & \text{d.l.camion.} 
\end{cases} $$
$$ t_{n, camion} \in \mathbb{R}^{+} \text{: Tiempo de llegada del vehículo } camion \text{ al nodo } n \in \text{NODOS} $$

In [24]:
# Variables de decisión
x = model.binary_var_dict([(nodo1, nodo2, camion) for nodo1 in NODOS for nodo2 in NODOS for camion in CAMIONES], name="x")
t = model.continuous_var_dict([(nodo, camion) for nodo in NODOS for camion in CAMIONES], name="t")

**Función Objetivo**
$$ \text{Minimizar } \sum_{camion \in \text{CAMIONES}} \sum_{n1 \in \text{NODOS}} \sum_{n2 \in \text{NODOS}} \text{DISTANCIAS}_{n1, n2} \cdot x_{n1, n2, camion} $$

In [25]:
# Función objetivo
model.minimize(model.sum(TIEMPO_VIAJE[(nodo1, nodo2)] * x[(nodo1, nodo2, camion)] for nodo1 in NODOS for nodo2 in NODOS for camion in CAMIONES))

**Restricciones**

1. **Prohibición de bucles:**
$$ \forall n \in \text{NODOS}, \forall camion \in \text{CAMIONES}: x_{n, n, camion} = 0 $$

2. **Asignación única de salidas para cada cliente:**
$$ \forall cliente \in \text{CLIENTES}: \sum_{n \in \text{NODOS}} \sum_{camion \in \text{CAMIONES}} x_{cliente, n, camion} = 1 $$

3. **Asignación única de entradas para cada cliente:**
$$ \forall cliente \in \text{CLIENTES}: \sum_{n \in \text{NODOS}} \sum_{camion \in \text{CAMIONES}} x_{n, cliente, camion} = 1 $$

4. **Visita de un camión desde el depósito a un cliente:**
$$ \forall camion \in \text{CAMIONES}: \sum_{cliente \in \text{CLIENTES}} x_{0, cliente, camion} \leq 1 $$

5. **Retorno de un camión del cliente al depósito:**
$$ \forall camion \in \text{CAMIONES}: \sum_{cliente \in \text{CLIENTES}} x_{cliente, 0, camion} \leq 1 $$

6. **Flujo de continuidad:**
$$ \forall camion \in \text{CAMIONES}, \forall n \in \text{NODOS}: \sum_{n' \in \text{NODOS}} x_{n', n, camion} = \sum_{n' \in \text{NODOS}} x_{n, n', camion} $$

7. **Restricciones de ventana de tiempo:**
$$ \forall cliente \in \text{CLIENTES}, \forall camion \in \text{CAMIONES}: a_{cliente} \leq t_{cliente, camion} \leq b_{cliente} $$

8. **Tiempo de llegada y servicio:**
$$ \forall cl1, cl2 \in \text{CLIENTES}, cl1 \neq cl2, \forall c \in \text{CAMIONES}: \text{Si } x_{cl1, cl2, c} = 1 \rightarrow t_{cl2, c} = t_{cl1, c} + \text{TIEMPO\_TRAVEL}_{cl1, cl2} + \text{TIEMPO\_SERVICIO}_{cl2}$$
$$ \forall cl1, cl2 \in \text{CLIENTES}, cl1 \neq cl2, \forall c \in \text{CAMIONES}: t_{cl2, c} \geq t_{cl1, c} + \text{TIEMPO\_TRAVEL}_{cl1, cl2} + \text{TIEMPO\_SERVICIO}_{cl2} - \mathcal{M}  \cdot (1 - x_{cl1, cl2, c}) $$


9. **Tiempo de llegada del depósito a los clientes:**
$$ \forall cl \in \text{CLIENTES}, \forall c \in \text{CAMIONES}: \text{Si } x_{0, cl, c} = 1 \rightarrow t_{cl, camion} =  \text{TIEMPO\_TRAVEL}_{0, cliente} + \text{TIEMPO\_SERVICIO}_{cl}$$
$$ \forall cl \in \text{CLIENTES}, \forall c \in \text{CAMIONES}: t_{cl, camion} \geq \text{TIEMPO\_TRAVEL}_{0, cliente} + \text{TIEMPO\_SERVICIO}_{cl} - 10000 \cdot (1 - x_{0, cl, c}) $$


In [26]:
# Restricciones
# 1. Prohibición de bucles
for nodo in NODOS:
    for camion in CAMIONES:
        model.add_constraint(x[nodo, nodo, camion] == 0)

# 2. Asignación única de salidas para cada cliente
for cliente in CLIENTES:
    model.add_constraint(model.sum(x[cliente, nodo, camion] for nodo in NODOS for camion in CAMIONES) == 1)

# 3. Asignación única de entradas para cada cliente
for cliente in CLIENTES:
    model.add_constraint(model.sum(x[nodo, cliente, camion] for nodo in NODOS for camion in CAMIONES) == 1)

for camion in CAMIONES:
    model.add_constraint(model.sum(x[0, cliente, camion] for cliente in CLIENTES) <= 1)
    model.add_constraint(model.sum(x[cliente, 0, camion] for cliente in CLIENTES) <= 1)

# 6. Flujo de continuidad
for camion in CAMIONES:
    for nodo in NODOS:
        model.add_constraint(model.sum(x[nodo_, nodo, camion] for nodo_ in NODOS) == model.sum(x[nodo, nodo_, camion] for nodo_ in NODOS))

# 7. Restricciones de ventana de tiempo
for cliente in CLIENTES:
    for camion in CAMIONES:
        model.add_constraint(t[cliente, camion] - TIEMPO_SERVICIO[cliente] >= VENTANA_TIEMPO[cliente]["mint"])
        model.add_constraint(t[cliente, camion] <= VENTANA_TIEMPO[cliente]["maxt"])
        
# 8. Tiempo de llegada y servicio
for cliente1 in CLIENTES:
    for cliente2 in CLIENTES:
        for camion in CAMIONES:
            if cliente1 != cliente2:
                # model.add_constraint(t[cliente2, camion] >= t[cliente1, camion] + TIEMPO_VIAJE[(cliente1, cliente2)] + TIEMPO_SERVICIO[cliente1] - 10000 * (1 - x[cliente1, cliente2, camion]))
                model.add_indicator(x[cliente1, cliente2, camion], t[cliente2, camion] == t[cliente1, camion] + TIEMPO_VIAJE[(cliente1, cliente2)] + TIEMPO_SERVICIO[cliente2], active_value = 1)

# 9. Tiempo de lltegada del depósito a los clientes
for cliente in CLIENTES:
    for camion in CAMIONES:
        # model.add_constraint(t[cliente, camion] >= TIEMPO_VIAJE[(0, cliente)] + 0 - 10000 * (1 - x[0, cliente, camion]))
        model.add_indicator(x[0, cliente, camion], t[cliente, camion] == TIEMPO_VIAJE[0, cliente] + TIEMPO_SERVICIO[cliente], active_value = 1)

In [27]:
# Resolviendo el modelo
# model.parameters.mip.limits.solutions = 1
model.set_time_limit(20)
solution = model.solve(log_output=True)


Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
CPXPARAM_TimeLimit                               20


Tried aggregator 2 times.
MIP Presolve eliminated 427 rows and 651 columns.
MIP Presolve modified 959 coefficients.
Aggregator did 959 substitutions.
Reduced MIP has 1482 rows, 3864 columns, and 13615 nonzeros.
Reduced MIP has 2443 binaries, 0 generals, 0 SOSs, and 2303 indicators.
Presolve time = 0.05 sec. (23.18 ticks)
Probing time = 0.09 sec. (23.69 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 1482 rows, 3864 columns, and 13615 nonzeros.
Reduced MIP has 2443 binaries, 0 generals, 0 SOSs, and 2303 indicators.
Presolve time = 0.01 sec. (9.28 ticks)
Probing time = 0.01 sec. (4.74 ticks)
Clique table members: 8333.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 12 threads.
Root relaxation solution time = 0.02 sec. (3.73 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

      0     0   

In [28]:
rutas = {}
cliente_camion = {}
for camion in CAMIONES:
    for cliente in CLIENTES:
        if x[0, cliente, camion].solution_value > 0.9:
            print(f"==== Ruta camión {camion} ====")
            ruta = [0, cliente]
            r = "0 -> " + str(cliente) + " -> "
            cliente_camion[cliente] = camion

            cliente_actual = cliente
            while x[cliente_actual, 0, camion].solution_value < 0.9:
                for siguiente_cliente in CLIENTES:
                    if x[cliente_actual, siguiente_cliente, camion].solution_value > 0.9:
                        break
                cliente_camion[siguiente_cliente] = camion
                ruta.append(siguiente_cliente)
                r += str(siguiente_cliente) + " -> "
                cliente_actual = siguiente_cliente
                

            ruta.append(0)
            r += "0"
            print(r)
            rutas[camion] = ruta
            
print("\n==== Arreglo de rutas ====")
print(rutas)


==== Ruta camión 1 ====
0 -> 11 -> 3 -> 12 -> 9 -> 16 -> 18 -> 0
==== Ruta camión 2 ====
0 -> 19 -> 7 -> 10 -> 17 -> 0
==== Ruta camión 3 ====
0 -> 5 -> 8 -> 13 -> 14 -> 4 -> 0
==== Ruta camión 5 ====
0 -> 15 -> 6 -> 1 -> 20 -> 2 -> 0

==== Arreglo de rutas ====
{1: [0, 11, 3, 12, 9, 16, 18, 0], 2: [0, 19, 7, 10, 17, 0], 3: [0, 5, 8, 13, 14, 4, 0], 5: [0, 15, 6, 1, 20, 2, 0]}


In [29]:
fig = go.Figure()

# RUTAS
for id_camion, ruta in rutas.items():
    coords_x = [COORDENADAS[nodo]["x"] for nodo in ruta]
    coords_y = [COORDENADAS[nodo]["y"] for nodo in ruta]
    tiempo_ruta = sum([TIEMPO_VIAJE[(ruta[i], ruta[i + 1])] for i in range(len(ruta) - 1)])

    fig.add_trace(
        go.Scatter(
            x=coords_x,
            y=coords_y,
            mode="lines",
            line=dict(color=px.colors.qualitative.G10[id_camion % len(px.colors.qualitative.G10)], width=2, dash="solid"),
            name=f"Ruta C{id_camion} - T: {tiempo_ruta:.2f}",
        )
    )

    # FLECHAS
    for i in range(len(ruta) - 1):
        mid_x = (coords_x[i] + coords_x[i + 1]) / 2
        mid_y = (coords_y[i] + coords_y[i + 1]) / 2
        fig.add_annotation(
            x=mid_x, y=mid_y,
            ax=mid_x - (coords_x[i + 1] - coords_x[i]) * 0.1, ay=mid_y - (coords_y[i + 1] - coords_y[i]) * 0.1,
            xref="x",
            yref="y",
            axref="x",
            ayref="y",
            showarrow=True,
            arrowhead=1,
            arrowsize=2,
            arrowwidth=1,
            arrowcolor=px.colors.qualitative.G10[id_camion % len(px.colors.qualitative.G10)],
        )

# DEPÓSITO
fig.add_trace(
    go.Scatter(
        x=[COORDENADAS[0]["x"]],
        y=[COORDENADAS[0]["y"]],
        mode="markers",
        name="Depósito",
        marker=dict(color="red", size=15),
    )
)

# CLIENTES
fig.add_trace(
    go.Scatter(
        x=[coord["x"] for coord in list(COORDENADAS.values())[1:]],
        y=[coord["y"] for coord in list(COORDENADAS.values())[1:]],
        mode="markers",
        name="Clientes",
        marker=dict(color="black", size=10),
    )
)

for cliente in CLIENTES:
    coord_cliente_x = COORDENADAS[cliente]["x"]
    coord_cliente_y = COORDENADAS[cliente]["y"]
    id_camion = cliente_camion[cliente]
    ventana = f"[{VENTANA_TIEMPO[cliente]['mint']}, {VENTANA_TIEMPO[cliente]['maxt']}]"
    t_llegada = t[cliente, id_camion].solution_value - TIEMPO_SERVICIO[cliente]
    t_salida = t[cliente, id_camion].solution_value
    fig.add_annotation(x=coord_cliente_x - 8, y=coord_cliente_y + 8, text=f"<b>C{cliente}:</b>  {ventana}<br />TI: {t_llegada:.2f} | TF: {t_salida:.2f}", font=dict(size=10), showarrow=False)

fig.update_layout(
    title=f"RECORRIDO TOTAL: {solution.objective_value:.2f}",
    xaxis_title="X",
    yaxis_title="Y",
    width=1200,
    height=1000,
    template="ggplot2",
)

fig.show()

In [33]:
i = 19

In [34]:
TIEMPO_VIAJE[0, i]

8.130190649671139

In [35]:
TIEMPO_SERVICIO[i] + TIEMPO_VIAJE[0, i]

12.130190649671139